In [1]:
%load_ext autoreload
%autoreload 2

from math import comb
from scipy.sparse.linalg import inv, aslinearoperator, lsqr

from reg_sr.utils import *
from reg_sr.losses import *
from reg_sr.regularizers import *
from reg_sr.firstOrderMethods import gradientDescent

import reg_sr

In [2]:
# pde = PhDExchange()
# g = pde.get_data(annot="c18basic")
# g = pde.get_data(annot="sector")
# g = pde.get_data(annot="stabbr")
# g.save("graph_for_debugging.gt", fmt="gt")
g = gt.load_graph("graph_for_debugging.gt")


In [3]:
# Just make the graph smaller for debugging
np.random.seed(0)
u = gt.GraphView(g, vfilt=lambda v: np.random.rand(1) > 0.8)

B, b = cast2sum_squares_form(u, alpha=1)
print(u)

<GraphView object, directed, with 41 vertices and 357 edges, 1 internal vertex property, edges filtered by (<EdgePropertyMap object with value type 'bool', for Graph 0x7f41f0587a00, at 0x7f40a5d9db70>, False), vertices filtered by (<VertexPropertyMap object with value type 'bool', for Graph 0x7f41f0587a00, at 0x7f40a5d9e020>, False), at 0x7f41f0587a00>


In [4]:
num_classes = len(set(np.array(list(u.vp["class"]))))
num_pairs_classes = comb(num_classes, 2)

sslc = sum_squared_loss_conj()
sslc.setup(u, alpha=1)

f = lambda x: sslc.evaluate(x)
grad = lambda x: sslc.prox(x)
prox = lambda x, t: same_mean_reg(tau=1).prox(x, t)
prox_fcn = lambda x: same_mean_reg(tau=1).evaluate(x)

x0 = np.random.rand(num_pairs_classes, 1)

In [5]:
Lip_c = sslc.find_Lipschitz_constant()
xNew, data = gradientDescent(
    f,
    grad,
    x0,
    prox=prox,
    prox_obj=prox_fcn,
    stepsize=Lip_c ** -1,
    printEvery=5000,
    maxIters=1e5,
    tol=1e-16,  # orig 1e-14
    saveHistory=True,
    linesearch=True,
    acceleration=False,
    restart=50
)

print(f"xNew: {xNew.T}")

Iter.  Objective Stepsize
-----  --------- --------
    0  -9.61e+01  2.33e-01
 5000  -8.27e+01  1.66e+00
10000  -8.27e+01  1.59e+00
15000  -8.27e+01  1.68e+00
20000  -8.27e+01  1.61e+00
25000  -8.27e+01  1.72e+00
30000  -8.27e+01  1.65e+00
35000  -8.27e+01  1.74e+00
40000  -8.27e+01  1.67e+00
45000  -8.27e+01  1.60e+00
50000  -8.27e+01  1.70e+00
55000  -8.27e+01  1.63e+00
60000  -8.27e+01  1.72e+00
65000  -8.27e+01  1.65e+00
70000  -8.27e+01  1.58e+00
75000  -8.27e+01  1.69e+00
80000  -8.27e+01  1.62e+00
85000  -8.27e+01  1.71e+00
90000  -8.27e+01  1.64e+00
95000  -8.27e+01  1.74e+00
100000  -8.27e+01  1.67e+00
100000  -8.27e+01  1.67e+00
==  Reached max iterations  ==
xNew: [[-1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]


In [11]:
ell = compute_ell(g, "class")

In [9]:
g.list_properties()

class          (vertex)  (type: string)
